In [12]:
from google.colab import files
uploaded = files.upload()

Saving credit_card_process_activities.csv to credit_card_process_activities (1).csv


In [14]:
import pandas as pd
from datetime import datetime

class ProcessMasterBuilder:
    def __init__(self, activity_df):
        self.activity_df = activity_df.copy()
        self.activity_df['Activity_Timestamp'] = pd.to_datetime(self.activity_df['Activity_Timestamp'])
        self.activity_df['Stage_Start_Timestamp'] = pd.to_datetime(self.activity_df['Stage_Start_Timestamp'])
        self.activity_df['Stage_End_Timestamp'] = pd.to_datetime(self.activity_df['Stage_End_Timestamp'])
        self.application_master = pd.DataFrame()
        self.stage_master = pd.DataFrame()
        self.build_stage_master()
        self.build_application_master()
        self.show_tables()

    def build_stage_master(self):
        stage_summary = []
        for (app_id, stage), group in self.activity_df.groupby(['Application_ID', 'Stage']):
            stage_start = group['Stage_Start_Timestamp'].min()
            stage_end = group['Stage_End_Timestamp'].max()
            tat_minutes = (stage_end - stage_start).total_seconds() / 60
            app_start = self.activity_df[self.activity_df['Application_ID'] == app_id]['Activity_Timestamp'].min()
            age_days = (stage_end - app_start).days
            stage_row = {
                'Application_ID': app_id,
                'Stage': stage,
                'Stage_Start': stage_start,
                'Stage_End': stage_end,
                'TAT_Minutes': tat_minutes,
                'Age_Days': age_days,
                'Risk_Grade': group['Risk_Grade'].iloc[-1],
                'UW_Decision': group['UW_Decision'].iloc[-1] if 'UW_Decision' in group.columns else None,
                'Stage_Status': group['Status_After_Activity'].iloc[-1],
                'Performed_By': ", ".join(group['Performed_By'].unique()),
                'Issues_Count': 0,
            }
            stage_summary.append(stage_row)
        self.stage_master = pd.DataFrame(stage_summary)

    def build_application_master(self):
        app_summary = []
        for app_id, group in self.activity_df.groupby('Application_ID'):
            app_start = group['Activity_Timestamp'].min()
            app_end = group['Activity_Timestamp'].max()
            total_tat_minutes = (app_end - app_start).total_seconds() / 60
            total_age_days = (datetime.now() - app_start).days
            app_row = {
                'Application_ID': app_id,
                'Product_Type': group['Product_Type'].iloc[0],
                'Channel': group['Channel'].iloc[0],
                'Application_Start': app_start,
                'Application_End': app_end,
                'Total_TAT_Minutes': total_tat_minutes,
                'Age_Days': total_age_days,
                'Final_Risk_Grade': group['Risk_Grade'].iloc[-1],
                'Final_UW_Decision': group['UW_Decision'].iloc[-1] if 'UW_Decision' in group.columns else None,
                'Application_Status': group['Status_After_Activity'].iloc[-1],
                'Performed_By': ", ".join(group['Performed_By'].unique()),
                'Issues_Count': 0,
            }
            app_summary.append(app_row)
        self.application_master = pd.DataFrame(app_summary)

    def show_tables(self, n=5):
        print("Stage Master (first rows):")
        print(self.stage_master.head(n))
        print("\nApplication Master (first rows):")
        print(self.application_master.head(n))


class TransactionProcessor:
    def __init__(self, transaction, app_master, stage_master):
        self.transaction = transaction
        self.app_master = app_master
        self.stage_master = stage_master
        self.issue_detected = False
        self.status_changed = False
        self.stage_changed = False

    def process(self):
        app_id = self.transaction['Application_ID']
        stage = self.transaction['Stage']

        # Check if application exists
        if app_id not in self.app_master['Application_ID'].values:
            self.add_new_application()
        else:
            self.update_application()

        # Check if stage exists
        if not ((self.stage_master['Application_ID'] == app_id) & (self.stage_master['Stage'] == stage)).any():
            self.add_new_stage()
        else:
            self.update_stage()

    def add_new_application(self):
        new_app = {
            'Application_ID': self.transaction['Application_ID'],
            'Product_Type': self.transaction['Product_Type'],
            'Channel': self.transaction['Channel'],
            'Application_Start': self.transaction['Activity_Timestamp'],
            'Application_End': self.transaction['Activity_Timestamp'],
            'Total_TAT_Minutes': 0,
            'Age_Days': 0,
            'Final_Risk_Grade': self.transaction['Risk_Grade'],
            'Final_UW_Decision': self.transaction.get('UW_Decision', None),
            'Application_Status': self.transaction['Status_After_Activity'],
            'Performed_By': self.transaction['Performed_By'],
            'Issues_Count': 0,
        }
        self.app_master = pd.concat([self.app_master, pd.DataFrame([new_app])], ignore_index=True)

    def add_new_stage(self):
        new_stage = {
            'Application_ID': self.transaction['Application_ID'],
            'Stage': self.transaction['Stage'],
            'Stage_Start': self.transaction['Stage_Start_Timestamp'],
            'Stage_End': self.transaction['Stage_End_Timestamp'],
            'TAT_Minutes': 0,
            'Age_Days': 0,
            'Risk_Grade': self.transaction['Risk_Grade'],
            'UW_Decision': self.transaction.get('UW_Decision', None),
            'Stage_Status': self.transaction['Status_After_Activity'],
            'Performed_By': self.transaction['Performed_By'],
            'Issues_Count': 0,
        }
        self.stage_master = pd.concat([self.stage_master, pd.DataFrame([new_stage])], ignore_index=True)

    def update_application(self):
        idx = self.app_master[self.app_master['Application_ID'] == self.transaction['Application_ID']].index[0]
        app_row = self.app_master.loc[idx]
        if app_row['Application_Status'] != self.transaction['Status_After_Activity']:
            self.app_master.at[idx, 'Application_Status'] = self.transaction['Status_After_Activity']
            self.status_changed = True
        if app_row['Final_Risk_Grade'] != self.transaction['Risk_Grade']:
            self.app_master.at[idx, 'Final_Risk_Grade'] = self.transaction['Risk_Grade']
        if 'Issue_Flag' in self.transaction and self.transaction['Issue_Flag']:
            self.app_master.at[idx, 'Issues_Count'] += 1
            self.issue_detected = True

    def update_stage(self):
        idx = self.stage_master[
            (self.stage_master['Application_ID'] == self.transaction['Application_ID']) &
            (self.stage_master['Stage'] == self.transaction['Stage'])
        ].index[0]
        stage_row = self.stage_master.loc[idx]
        if stage_row['Stage_Status'] != self.transaction['Status_After_Activity']:
            self.stage_master.at[idx, 'Stage_Status'] = self.transaction['Status_After_Activity']
            self.status_changed = True
        if stage_row['Risk_Grade'] != self.transaction['Risk_Grade']:
            self.stage_master.at[idx, 'Risk_Grade'] = self.transaction['Risk_Grade']
        if 'Issue_Flag' in self.transaction and self.transaction['Issue_Flag']:
            self.stage_master.at[idx, 'Issues_Count'] += 1
            self.issue_detected = True

In [15]:
builder = ProcessMasterBuilder(df)

for i, txn in df.iterrows():
    processor = TransactionProcessor(txn, builder.application_master, builder.stage_master)
    processor.process()

builder.show_tables()

Stage Master (first rows):
  Application_ID         Stage         Stage_Start           Stage_End  \
0         APP001            DI 2025-09-10 05:14:00 2025-09-10 11:23:00   
1         APP001        Dedupe 2025-09-09 14:06:00 2025-09-09 20:00:00   
2         APP001          PDOC 2025-09-10 03:23:00 2025-09-10 05:04:00   
3         APP001      Sourcing 2025-09-09 07:00:00 2025-09-09 13:10:00   
4         APP001  Underwriting 2025-09-09 20:32:00 2025-09-10 02:25:00   

   TAT_Minutes  Age_Days Risk_Grade UW_Decision Stage_Status  \
0        369.0         1     Medium     Decline     Complete   
1        354.0         0     Medium        Hold      On Hold   
2        101.0         0       High     Decline     Rejected   
3        370.0         0     Medium        Hold      On Hold   
4        353.0         0       High        Hold      Pending   

        Performed_By  Issues_Count  
0     Team A, Team B             0  
1       Team A, Auto             0  
2  Third Party, Auto            